## Установка и загрузка библиотек

In [1]:
!pip3 install python-rake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip3 install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip3 install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import io
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from summa import keywords
from keybert import KeyBERT

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Информация о мини-корпусе и ключевых словах в нём

В качестве мини-корпуса с разметкой ключевых слов были использованы материалы ресурса postnauka.ru (ссылки на использованные страницы находятся в датасете). Ключевые слова на "Постнауке" представляют собой скорее список тегов-категорий, которые позволяют классифицировать тексты на портале. 

Загрузка мини-корпуса:

In [6]:
url = 'https://github.com/asyakarysheva/NLP_2022/blob/main/hw1/hw1_corpus.xlsx?raw=true'
corpus = pd.read_excel(url) 

Можно посчитать объём корпуса в текстах:

In [7]:
len(corpus)

6

И в токенах:

In [8]:
tokenizer = RegexpTokenizer(r'\w+')
texts = corpus['text'].apply(tokenizer.tokenize).tolist()
tokens_number = 0
for text in texts:
    tokens_number += len(text)
tokens_number

5152

Вот как выглядит представленный на сайте набор ключевых слов для нулевого текста из мини-корпуса:

In [9]:
corpus['key_words_standard'][0]

'ПСИХОЛОГИЯ, МОЗГ, КОГНИТИВНАЯ ПСИХОЛОГИЯ, ВНИМАНИЕ, ВОСПРИЯТИЕ, СЛУХ, ПСИХОЛОГИЧЕСКИЙ ЭФФЕКТ, КОГНИТИВНЫЕ НАУКИ, ЖУРНАЛ'

## Самостоятельная разметка ключевых слов, составление эталона разметки

Так выглядят ключевые слова, выделенные мной, для нулевого текста в выборке:

In [10]:
corpus['key_words'][0]

'эффект коктейльной вечеринки, бинауральность, селективное внимание, субъективно важные сообщения, фильтр, ослабление, фоновый шум'

Насколько ключевые слова с "Постнауки" пересекаются с моими ключевыми словами?

In [11]:
key_words_standard = []
for key_word_set in list(corpus['key_words_standard']):
    key_word_set = key_word_set.lower()
    key_words_split = key_word_set.split(', ')
    key_words_standard.append(key_words_split)

key_words = []
for key_word_set in list(corpus['key_words']):
    key_word_set = key_word_set.lower()
    key_words_split = key_word_set.split(', ')
    key_words.append(key_words_split)

In [12]:
intersections = []
for key_word_standard, key_word in zip(key_words_standard, key_words):
    intersections.append(list(set(key_word_standard) & set(key_word)))
intersections

[[], ['слух'], ['культура'], [], [], []]

Пересечений крайне мало – в ключевых словах всего для двух текстов совпадает только одно ключевое слово; ключевые слова для остальных текстов не пересекаются. Можно предположить, что почти полное отсутствие пересечений связно с разными представлениями о ключевых словах у редакторов "Постнауки" и у меня. Кроме того, в ключевых словах, выделенных мной, встречается больше биграмм и триграмм.

В качестве эталона разметки выбрано объединение ключевых слов:

In [13]:
reference = []
for key_word_standard, key_word in zip(key_words_standard, key_words):
    in_key_word_standard = set(key_word_standard)
    in_key_word = set(key_word)

    in_keyword_but_not_in_standard = in_key_word - in_key_word_standard

    result = key_word_standard + list(in_keyword_but_not_in_standard)
    reference.append(result)

Ключевые слова для текстов выглядят так:

In [14]:
reference

[['психология',
  'мозг',
  'когнитивная психология',
  'внимание',
  'восприятие',
  'слух',
  'психологический эффект',
  'когнитивные науки',
  'журнал',
  'эффект коктейльной вечеринки',
  'ослабление',
  'бинауральность',
  'селективное внимание',
  'субъективно важные сообщения',
  'фильтр',
  'фоновый шум'],
 ['биология',
  'технологии',
  'информационные технологии',
  'слух',
  'естественные науки',
  'журнал',
  'телефон',
  'звук',
  'заушный аппарат',
  'диапазон',
  'слуховой аппарат',
  'речь',
  'внутриканальный аппарат',
  'усилитель',
  'ухо',
  'микрофон'],
 ['культура',
  'массовая культура',
  'народная культура',
  'мода',
  'молодежь',
  'гуманитарные науки',
  'журнал',
  'комфорт',
  'юмор',
  'красота',
  'уродливые вещи',
  'норма',
  'устойчивость',
  'пластичность гендера',
  'расслабленность',
  'полифония',
  'поп-культура',
  'уродливая мода'],
 ['социология',
  'поведение человека',
  'социология повседневности',
  'эксперимент',
  'социальные науки',
  

## Методы извлечения ключевых слов

### Предобработка текстов

Тексты нужно нормализовать.

In [15]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [16]:
# Нужно создать список с текстами:
corpus_texts = corpus['text'].tolist()

In [17]:
normalized_texts = []
for text in corpus_texts:
    normalized_text = normalize_text(text)
    normalized_texts.append(normalized_text)

### RAKE

In [18]:
# Загрузка стоп-слов
stop = stopwords.words('russian')
# Инициализируем анализатор списком стоп-слов
rake = RAKE.Rake(stop)

In [19]:
key_words_RAKE = []
for text in normalized_texts:
    kw_list = rake.run(text, maxWords=3, minFrequency=2)  # в эталоне разметки есть триграммы, поэтому maxWords=3
    key_words_RAKE.append(kw_list) 

In [20]:
key_words_RAKE

[[('обращать внимание', 4.0),
  ('информация', 2.3),
  ('который', 2.0),
  ('голос', 1.6666666666666667),
  ('нужный', 1.6666666666666667),
  ('модель', 1.6666666666666667),
  ('шум', 1.4),
  ('канал', 1.1666666666666667),
  ('кроме', 1.0),
  ('проблема', 1.0),
  ('мозг', 1.0),
  ('основа', 1.0),
  ('—', 0)],
 [('слуховой аппарат уменьшаться', 8.11111111111111),
  ('который помещаться', 4.619047619047619),
  ('который использоваться', 4.285714285714286),
  ('который работать', 4.285714285714286),
  ('поздний появиться', 4.0),
  ('аппарат', 2.3333333333333335),
  ('телефон', 1.75),
  ('усилитель', 1.5),
  ('ухо', 1.2),
  ('это', 1.1666666666666667),
  ('принцип', 1.0),
  ('состоять', 1.0),
  ('микрофон', 1.0),
  ('смартфон', 1.0),
  ('разбор', 1.0),
  ('человек', 1.0),
  ('уха', 1.0),
  ('проводы', 1.0),
  ('сначала', 1.0)],
 [('свой тело', 4.333333333333334),
  ('иметь дело', 4.0),
  ('2010-й год', 3.75),
  ('одежда', 2.0),
  ('это', 2.0),
  ('который', 1.8333333333333333),
  ('сегодня

### TextRank

In [21]:
key_words_TextRank = []
for text in normalized_texts:
    kw_list = keywords.keywords(text, language='russian', additional_stopwords=stop, scores=True)
    key_words_TextRank.append(kw_list)

In [22]:
key_words_TextRank

[[('который', 0.28715353780680797),
  ('внимание', 0.24431789597625955),
  ('человек', 0.2138406081568353),
  ('способный', 0.18142269623376042),
  ('поток информация', 0.151842742209668),
  ('шум', 0.14473132589623636),
  ('обстановка способность', 0.13865658496901062),
  ('ухо', 0.13086097622112675),
  ('уха', 0.13086097622112675),
  ('сигнал', 0.11952179609754597),
  ('механизм', 0.11889071372317525),
  ('определять', 0.11797153292044313),
  ('определяться', 0.11797153292044313),
  ('система мочь', 0.11174934523542433),
  ('распознавать', 0.10308362376812838),
  ('распознаваться', 0.10308362376812838),
  ('предложить', 0.09779786185630669),
  ('источник звук', 0.09760644069198077),
  ('начинать', 0.09723761944658492),
  ('канал', 0.0943518586078879),
  ('фильтр', 0.09308537266770685),
  ('основный слуховой', 0.09091348368416723),
  ('шумовой', 0.08943625233072125),
  ('свой исследование', 0.08770036404931611),
  ('разный стимул', 0.08691311045294514),
  ('трудный', 0.085997787903882

### KeyBERT

In [23]:
kw_model = KeyBERT('clips/mfaq')

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [24]:
key_words_keyBERT = []
for text in normalized_texts:
    kw_list = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words=stop, top_n = 10)
    key_words_keyBERT.append(kw_list)

In [25]:
key_words_keyBERT

[[('анализировать звуковой', 0.9763),
  ('слышать', 0.9762),
  ('вслух слышать определённый', 0.9761),
  ('анализировать звуковой информация', 0.9754),
  ('узнавать голос', 0.975),
  ('звуковой информация определять', 0.9748),
  ('голова знакомый звук', 0.9747),
  ('звук шум влиять', 0.9745),
  ('распознавать слуховой', 0.9743),
  ('шумовой стимул достигать', 0.9742)],
 [('устроить слуховой аппарат', 0.9781),
  ('слуховой аппарат устроить', 0.9764),
  ('устройство слуховой аппарат', 0.9751),
  ('устроить слуховой', 0.9747),
  ('сделать слуховой аппарат', 0.9747),
  ('создать электромеханический слуховой', 0.9743),
  ('современный слуховой аппарат', 0.9742),
  ('устройство слуховой', 0.9734),
  ('слуховой аппарат это', 0.973),
  ('аппарат устроить принцип', 0.9728)],
 [('сегодняшний сознательный модник', 0.9752),
  ('fashion уродливый мода', 0.9747),
  ('мода понятие красота', 0.9743),
  ('красота модный идеал', 0.9743),
  ('культуролог людмила', 0.9738),
  ('модный идеал жить', 0.9734)

## Морфологические шаблоны и фильтрация ключевых слов

Были выбраны несколько морфологических шаблонов для ключевых слов (шаблонам соответствует подавляющее большинство ключевых слов из эталона); в скобках указаны соответствующие им ключевые слова из эталона: 
- Noun ("психология"),
- Adj + Noun ("уродливые вещи"), 
- Noun + Noun ("восприятие времени"),
- Adj + Adj + Noun ("стэнфордский зефирный эксперимент"),
- Adv + Adj + Noun ("субъективно важные сообщения").

In [26]:
def POS_tagging(key_words):
    key_words_POS = []
    for item in key_words:
        new_item = []
        for i in item:
            new_tokens = []
            tokens = simple_word_tokenize(i[0])
            for token in tokens:
                p = m.parse(token)[0]
                POS_tag = p.tag.POS
                p_POS_tag = (token + '_' + str(POS_tag).strip())
                new_tokens.append(p_POS_tag)
            new_item.append(new_tokens)
        key_words_POS.append(new_item)

    return key_words_POS

In [27]:
key_words_RAKE_POS = POS_tagging(key_words_RAKE)
key_words_TextRank_POS = POS_tagging(key_words_TextRank)
key_words_keyBERT_POS = POS_tagging(key_words_keyBERT)

После того, как ключевые слова размечены по частям речи, можно оставить среди них только те, что соответствуют шаблонам:

In [28]:
def filter_key_words_POS(key_words):
    filtered_items = []
    for item in key_words:
        for i in item:
            if len(i) == 1 and "_NOUN" in str(i):
                filtered_items.append(i)
            if len(i) == 2 and "_ADJF" in str(i[0]) and "_NOUN" in str(i[1]):
                filtered_items.append((i[0], i[1]))
            if len(i) == 2 and "_NOUN" in str(i[0]) and "_NOUN" in str(i[1]):
                filtered_items.append((i[0], i[1]))
            if len(i) == 3 and "_ADJF" in str(i[0]) and "_ADJF" in str(i[1]) and "_NOUN" in str(i[2]):
                filtered_items.append((i[0], i[1], i[2]))
            if len(i) == 3 and "_ADV" in str(i[0]) and "_ADJF" in str(i[1]) and "_NOUN" in str(i[2]):
                filtered_items.append((i[0], i[1], i[2]))
    return filtered_items

In [29]:
key_words_RAKE_filtered = filter_key_words_POS(key_words_RAKE_POS)
key_words_TextRank_filtered = filter_key_words_POS(key_words_TextRank_POS)
key_words_keyBERT_filtered = filter_key_words_POS(key_words_keyBERT_POS)

Аналогичные фильтры нужно применить и к эталону (в нём есть ключевые слова, не соответствующие выделенным морфологическим шаблонам):

In [30]:
reference_POS = []
for item in reference:
    new_item = []
    for i in item:
        new_tokens = []
        tokens = simple_word_tokenize(i)
        for token in tokens:
            lemma = m.parse(token)[0].normal_form  # нужно лемматизировать ключевые слова
            p = m.parse(token)[0]
            POS_tag = p.tag.POS
            p_POS_tag = (lemma + '_' + str(POS_tag).strip())
            new_tokens.append(p_POS_tag)
        new_item.append(new_tokens)
    reference_POS.append(new_item)

In [31]:
reference_filtered = []
for item in reference_POS:
    for i in item:
        if len(i) == 1 and "_NOUN" in str(i):
            reference_filtered.append(i)
        if len(i) == 2 and "_ADJF" in str(i[0]) and "_NOUN" in str(i[1]):
            reference_filtered.append((i[0], i[1]))
        if len(i) == 2 and "_NOUN" in str(i[0]) and "_NOUN" in str(i[1]):
            reference_filtered.append((i[0], i[1]))
        if len(i) == 3 and "_ADJF" in str(i[0]) and "_ADJF" in str(i[1]) and "_NOUN" in str(i[2]):
            reference_filtered.append((i[0], i[1], i[2]))
        if len(i) == 3 and "_ADV" in str(i[0]) and "_ADJF" in str(i[1]) and "_NOUN" in str(i[2]):
            reference_filtered.append((i[0], i[1], i[2]))

## Оценка ключевых слов

### Без шаблонов

Прежде всего нужно лемматизировать эталон.

In [32]:
reference_normalized = []
for item in reference:
    new_item = []
    for i in item:
        new_tokens = []
        tokens = simple_word_tokenize(i)
        for token in tokens:
            lemma = m.parse(token)[0].normal_form
            new_tokens.append(lemma)
        new_item.append(new_tokens)
    reference_normalized.append(new_item)

Ключевые слова нужно привести к такому же формату, в каком представлен эталон:

In [33]:
def format_key_words(key_words):
    key_words_new = []
    for item in key_words:
        new_item = []
        for i in item:
            new_tokens = []
            tokens = simple_word_tokenize(i[0])
            for token in tokens:
                new_tokens.append(token)
            new_item.append(new_tokens)
        key_words_new.append(new_item)
    
    return key_words_new

In [34]:
key_words_RAKE_formatted = format_key_words(key_words_RAKE)
key_words_TextRank_formatted = format_key_words(key_words_TextRank)
key_words_keyBERT_formatted = format_key_words(key_words_keyBERT)

В очередной раз форматирую данные, чтобы можно было удобнее считать их пересечения:

In [35]:
def format_for_metrics(key_words):
    new_key_words = []
    for l in key_words:
        for item in l:
            new_item = ' '.join(item)
            new_key_words.append(new_item)

    return new_key_words

In [36]:
key_words_RAKE_for_metrics = format_for_metrics(key_words_RAKE_formatted)
key_words_TextRank_for_metrics = format_for_metrics(key_words_TextRank_formatted)
key_words_keyBERT_for_metrics = format_for_metrics(key_words_keyBERT_formatted)
reference_for_metrics = format_for_metrics(reference_normalized)

In [37]:
def compute_metrics(reference, key_words):
    TP = 0
    FN = 0
    FP = 0
    reference_set = set(reference)
    key_words_set = set(key_words)
    intersections = reference_set.intersection(key_words_set)
    TP += len(intersections)
    in_reference_but_not_in_keywords = reference_set.difference(key_words_set)
    FN += len(in_reference_but_not_in_keywords)
    in_keywords_but_not_in_reference = key_words_set.difference(reference_set)
    FP += len(in_keywords_but_not_in_reference)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    if precision + recall > 0:
        f1 = (2 * precision * recall) / (precision + recall)
        print('F-мера', f1)
    else:
        print('Нельзя посчитать F-меру')
    print('Точность:', precision)
    print('Полнота:', recall)

In [38]:
print('Для ключевых слов, полученных с помощью RAKE: \n')
compute_metrics(reference_for_metrics, key_words_RAKE_for_metrics)

print('\n Для ключевых слов, полученных с помощью TextRank: \n')
compute_metrics(reference_for_metrics, key_words_TextRank_for_metrics)

print('\n Для ключевых слов, полученных с помощью keyBERT: \n')
compute_metrics(reference_for_metrics, key_words_keyBERT_for_metrics)

Для ключевых слов, полученных с помощью RAKE: 

F-мера 0.08429118773946362
Точность: 0.05945945945945946
Полнота: 0.14473684210526316

 Для ключевых слов, полученных с помощью TextRank: 

F-мера 0.1566265060240964
Точность: 0.1015625
Полнота: 0.34210526315789475

 Для ключевых слов, полученных с помощью keyBERT: 

F-мера 0.029411764705882353
Точность: 0.03333333333333333
Полнота: 0.02631578947368421


### С учётом шаблонов

In [39]:
def format_for_metrics_POS(key_words):
    new_key_words = []
    for item in key_words:
        new_item = ' '.join(item)
        new_key_words.append(new_item)

    return new_key_words

In [40]:
key_words_RAKE_for_metrics_POS = format_for_metrics_POS(key_words_RAKE_filtered)
key_words_TextRank_for_metrics_POS = format_for_metrics_POS(key_words_TextRank_filtered)
key_words_keyBERT_for_metrics_POS = format_for_metrics_POS(key_words_keyBERT_filtered)
reference_for_metrics_POS = format_for_metrics_POS(reference_filtered)

In [41]:
print('Для ключевых слов с шаблонами, полученных с помощью RAKE: \n')
compute_metrics(reference_for_metrics_POS, key_words_RAKE_for_metrics_POS)

print('\n Для ключевых слов с шаблонами, полученных с помощью TextRank: \n')
compute_metrics(reference_for_metrics_POS, key_words_TextRank_for_metrics_POS)

print('\n Для ключевых слов с шаблонами, полученных с помощью keyBERT: \n')
compute_metrics(reference_for_metrics_POS, key_words_keyBERT_for_metrics_POS)

Для ключевых слов с шаблонами, полученных с помощью RAKE: 

F-мера 0.1456953642384106
Точность: 0.13414634146341464
Полнота: 0.15942028985507245

 Для ключевых слов с шаблонами, полученных с помощью TextRank: 

F-мера 0.24468085106382975
Точность: 0.19327731092436976
Полнота: 0.3333333333333333

 Для ключевых слов с шаблонами, полученных с помощью keyBERT: 

F-мера 0.05128205128205128
Точность: 0.2222222222222222
Полнота: 0.028985507246376812


## Ошибки автоматического выделения ключевых слов 


### RAKE

При автоматическом выделении ключевых слов с помощью метода RAKE возникает несколько проблем:

- как ключевые слова выделяются слова, близкие к стоп-словам, или биграммы, в состав которых входят эти слова (например, "который", "который помещаться");

- часто выделяются глаголы и глагольные группы, которые при ручном аннотировании скорее не будут отнесены к ключевым словам (например, "обращать внимание");

- среди ключевых слов оказывается довольно много числительных, которые кажутся полезными, например, для задачи суммаризации текста, но скорее не представляются "прототипическими" ключевыми словами (то есть такими, которые были бы отмечены при ручном аннотировании текстов) (например, "2018 год").

Можно предположить, что многие из указанных проблем могут решиться добавлением нескольких дополнительных слов в список стоп-слов, а также, например, удалением из текстов отдельных частей речи (глаголов, числительных).

### TextRank

Среди недостатков этого метода выделения ключевых слов можно отметить следующее:

- для каждого текста выделяется большое количество ключевых слов, многие из которых являются глаголами и не кажутся значимыми для текста (например, "казаться", "подтвердиться", "дать");

- встречаются слова, близкие к стоп-словам (например, "это", "который");

- среди ключевых слов встречается небольшое количество биграмм или триграмм.

Для того, чтобы избежать этих проблем, можно расширить список стоп-слов, а также отбирать топ-n ключевых слов для текста (плюс можно удалить из текстов слова определённых частей речи, например, глаголы).

С другой стороны, кажется, обширный список ключевых слов с большим количеством глаголов может считаться скорее плюсом в контексте решения задачи суммаризации текста.

### keyBERT

К недостаткам этого метода выделения ключевых слов может быть отнесено следующее:

- если модели предлагается выделить униграммы, биграммы и триграммы, модель выделяет преимущественно триграммы, причём иногда практически идентичные (например, "устроить слуховой аппарат", "сделать слуховой аппарат");

- при дефолтных параметрах в состав триграмм попадает довольно много стоп-слов (например, "он", "что");

- среди ключевых слов и фраз встречается много глаголов.

Возможно, первая проблема связана с небольшим объёмом и количеством текстов, которые подаются на вход модели (может быть, если бы тексты были объёмнее и их было бы больше, ключевые слова оказались бы более разнообразными). Вторая проблема может быть решена с помощью расширения списка стоп-слов. Третий недостаток может быть устранён, если удалить из текстов глаголы.